# Forward Backward et FISTA.

Dans ce TP, nous allons utiliser l'algorithme dit Forward Backward et sa version accélérée appelée FISTA pour résoudre différents problèmes d'optimisation associés à de l'inpainting et du débruitage. Nous travaillerons sur les deux images de référence du TP précédent mais à nouveau, vous pouvez utiliser d'autres images.



In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests
import boto3
import os

In [ ]:
#on installe ceci afin de coder les expressions en latex
pip install nbconvert

In [ ]:
s3_endpoint_url = 'https://object-rook-ceph.apps.math.cnrs.fr/'
s3_access_key_id = '9F7EB8YBUWXDV7A4IZYW' # le contenu de secrets/dossal
s3_secret_access_key = 'skV01Eei5M3xVOxROIDr3qymYhWtkrxPpMyj8nwb' # le contenu de secrets/dossal
s3_bucket = 'signal-image'
s3 = boto3.client('s3',
                  '',
                  endpoint_url = s3_endpoint_url,
                  aws_access_key_id = s3_access_key_id,
                  aws_secret_access_key = s3_secret_access_key)
Data=["Lenna.jpg","Canaletto.jpeg","MinotaureBruite.jpeg","Cartoon.jpg"]
if not os.path.isfile('Lenna.jpg'):
    for filenames in Data:  
        s3.download_file(s3_bucket,filenames,filenames)
def chargeData(name):
    if name=='Lenna':
        res=np.array(Image.open("Lenna.jpg")).astype(float)
    if name=='Canaletto':
        res=np.array(Image.open("Canaletto.jpeg")).astype(float)
    if name=='Minotaure':
        res=np.array(Image.open("MinotaureBruite.jpeg")).astype(float)  
    if name=='Cartoon':
        res=np.array(Image.open("Cartoon.jpg")).astype(float) 
    return res
options1=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None)
options2=dict(width=700,height=400,xaxis=None,yaxis=None,toolbar=None)

In [ ]:
im2=chargeData('Lenna')
im=chargeData('Canaletto')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options),hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
#Blocks2,Piece,im,im2,Mi=np.load('dataTP.npy')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options),hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
def PSNR(I,Iref):
    temp=I.ravel()
    tempref=Iref.ravel()
    NbP=I.size
    EQM=np.sum((temp-tempref)**2)/NbP
    b=np.max(np.abs(tempref))**2
    return 10*np.log10(b/EQM)

# Inpainting

Pour le problème d'inpainting on va supposer qu'on dispose d'observations $y=Mx^0+b$ qui sont dégradées par un opérateur de masquage $M$ et éventuellement par un bruit additif $b$. On va chercher à estimer $x^0$ en minimisant une fonctionnelle de la forme 
\begin{equation}\label{Eq1}
\frac{1}{2}\Vert Mx-y\Vert^2+\lambda \Vert Tx\Vert_1
\end{equation}
où $T$ est une transformée dans laquelle on sait que l'image $x^0$ est parcimonieuse (typiquement une bonne base d'ondelettes comme db2). Pour utiliser le FB, il faut calculer le gradient du terme d'attache aux données et l'opérateur proximal de $\Vert Tx\Vert_1$. Vous aurez de l'aide plus loin dans le notebook pour cette partie.

Si le bruit est nul, c'est-à-dire si $y=Mx^0$, où $M$ est un opérateur de masquage, il serait même plus intéressant de déterminer une solution du problème d'optimisation suivant :
\begin{equation}\label{Eq2}
\underset{x}{\min}\Vert Tx\Vert_1\text{ sous la contrainte }y=Mx  
\end{equation}
Ce problème d'optimisation ne peut être résolu directement par par l'algorithme Forward-Backward (FB) car on ne peut pas le formuler comme une minimsaition d'une somme de deux fonctions dont l'onde est différentiable à gradient Lipschitz. 
De plus, en pratique, il est souvent illusoire de penser que les données $y$ ne sont pas corrompues par un bruit, aussi petit soit il. 

On peut démontrer que les solutions de \eqref{Eq1} convergent ver une solution de \eqref{Eq2} quand $\lambda$ tend vers 0. L'avantage de cette formulation \eqref{Eq1} est qu'elle peut être résolu par FB. 
Résoudre \eqref{Eq1} revient ainsi à la fois à inpainter et à régulariser. Plus $\lambda$ est grand plus on régularise. Dans la pratique, en l'absence de bruit, on a intérêt à choisir une petite valeur de $\lambda$ dans la formulation \eqref{Eq1}. Le problème dun tel choix est que ça ralentit l'algorithme : i.e il faut un plus grand nombre d'itérations pour atteindre un résultat comparable si on prend le même point de départ. Quand on utilise ces méthodes, les choix de $\lambda$, du nombre d'itérations et du point de départ de l'algorithme sont importants.

# Rappels sur la descente de gradient explicite et sur Forward-Backward (FB)

Si $F=f+g$ est une fonction convexe composite, somme de deux fonctions convexes, $f$ différentiable à gradient 
$L$-Lipschitz et $g$ une fonction convexe dont on sait calculer l'opérateur proximal, l'algorithme Forward-Backward s'écrit 
$$x_{n+1}=prox_{hg}(x_n-h\nabla f(x_n))=Tx_n\quad \text{ avec }T:=prox_{hg}\circ (Id-h\nabla f)$$

On peut montrer que la suite de terme général $F(x_n)-F(x^*)$ est décroissante et de plus que 
$$F(x_n)-F(x^*)\leqslant \frac{2\Vert x_0-x^*\Vert^2}{hn}$$
Cette vitesse en $\frac{1}{n}$ est optimale au sens où il n'est pas possible de trouver des bornes qui décroissent en $\frac{1}{n^{\delta}}$ avec $\delta>1$ pour toutes les fonctions convexes. Cela étant on peut montrer que si $h<\frac{1}{L}$ on a en fait 
$$F(x_n)-F(x^*)=o\left(\frac{1}{n}\right)$$
et que cette vitesse est même géométrique si la fonction $f$ est fortement convexe, dans le cas différentiable. 


## Remarques sur les algorithmes itératifs en général et FB en particulier.

Lorsqu'on utilise un algorithme itératif qui dépend d'un ou plusieurs paramètres, il faut être vigilent pour évaluer ses performances. Il est important de s'assurer d'une manière ou d'une autre des points suivants :

1) Vérifier qu'on est proche de la convergence... c'est-à-dire qu'on a fait suffisamment d'itérations. Ce nombre peut varier en fonction des paramètres de l'agorithme et du point initial $x_0$ choisi pour construire la suite.

Je vous invite pour cela à afficher la courbe de la valeur de la fonctionnelle et à prendre un peu de recul sur ce que vous observez. S'il reste des pixels noirs qui sont manifestement des résidus du masque ou si la fonctionnelle décroit encore fortement, c'est qu'il faut sans doute augmenter le nombre d'itérations. D'une manière générale, observer les artefacts (défauts) de l'image reconstruite peut donner des éléments de réponses.
De plus, observer la courbe de la valeur de la fonctionnelle peut parfois indiquer qu'on aurait pu effectuer moins d'itérations pour un résulat comparable. 

2) Explorer les valeurs des paramètres. Il se peut que certaines valeurs de paramètres fournissent des résultats pertinents, d'autres non. Avant de trancher sur le caractère efficace d'une méthode, il faut avoir pris le temps de vérifier que le choix des paramètres est correct. Les paramètres peuvent être ceux de la fonctionnelle à minimiser, comme le $\lambda$ de \eqref{Eq1} mais aussi des paramètres internes à l'algorithme comme le pas de descente par exemple. Le fait de faire un plan d'expériences dans ce cas sur des données tests peut être un bon moyen de déterminer des paramètres pertinents. 

On peut noter que la valeur de la fonctionnelle ne fait pas tout, en effet si on veut comparer différentes valeur de $\lambda$, c'est plutôt le PSNR de la reconstruction qui sera un critère pertinent. On ne peut pas se passer de réfléchir... 

3) Bien choisir la valeur initiale de la suite. Les algorithmes itératifs construisent de manière séquentielle des images qui s'approchent du minimiseur de la fonctionnelle. Le choix du point de départ de la suite peut être crucial.
Si on ne dispose d'aucune information a priori, on peut partir d'une image constante à 0 ou d'une image raisonnable.
Par exemple, de l'image masquée si on fait de l'inpainting, ou de l'image bruitée si on fait du débruitage.

On peut souvent distinguer deux phases dans l'optimisation de la fonctionnelle, une première qui permet de passer de l'image initiale vers une image raisonnable, une approximation grossière du résultat, puis une seconde qui permet de passer de cette première approximation grossière à une image très proche du résultat final souhaité.
Il est difficile d'esquiver la seconde phase mais on peut réduire drastiquement la première en proposant une image initiale déjà très raisonnable. On peut par exemple lancer un algorithme classique rapide, mais pas très précis pour donner une image initiale meilleure qu'une initialisation aléatoire ou naïve. 
Dans le cas du débruitage, on peut par exemple filtrer l'image originale ou la seuiller en ondelettes. Dans le cas de l'inpainting on peut appliquer localement un filtre médian. C'est-à-dire on découpe l'image en blocs de taille fixe (8 par 8 par exemple) et sur chaque bloc on estime la valeur médianne d'intensité lumineuse parmi les valeurs non masquées et on remplace, sur chaque bloc, les valeurs inconnues par cette valeur. C'est primitif mais rapide et ça permet d'avoir une première estimation raisonnable.

D'une manière générale, si vous voulez comparer plusieurs méthodes, avec des paramètres ou des bases d'ondelettes différentes, il est pertinent de calculer le PSNR des images reconstruites. 
C'est une manière d'obtenir un critère un minimum objectif.

## Début de notre TP :

Avant de nous lancer dans les algorithmes d'optimisation, commençons par expliciter les calculs de gradient et de l'opérateur proximale dont nous allons nous servir pour la suite. Notre objectif est de minimiser :

$$
\hat{x} = argmin_{x \in \mathbb{R}^n } \frac{1}{2} {\lVert Mx - y \rVert}²_{2} + \lambda {\lVert Tx \rVert}_{1} := F(x)
$$

On note :
$$ f(x) =\frac{1}{2} {\lVert Mx - y \rVert}²_{2}$$ 

et 

$$ g(x) =\lambda {\lVert Tx \rVert}_{1}  $$

$f(x)$ étant une fonction différenciable, on peut calculer son gradient tel que :
    $$\nabla f(x) = M^T(Mx-y) = M(Mx-y)$$
    
$g(x)$ n'étant que convexe on calcule son opérateur proximal tel que : 
$$ prox_{h°g}(x) = T*soft_h(Tx) $$ où $soft$ représente le seuillage doux.

De plus, pour que FB soit un algorithme de descente, il nous faut $h< \frac{1}{L}$, où $h$ correspond au pas et $L$ à la constante de Lipschitz de la fonction $f$. Or $L=1$ ici car :
$${\lVert M \rVert}= 1$$d'où $${\lVert M \rVert}²= 1$$

Ainsi notre condition sur $h$ est : $h<1$ pour que l'algorithme soit un algorithme de descente.

## Mise en place de FB pour l'inpainting.

On pourra commencer dans un premier temps par des observations non bruitées et effectuer seulement un masquage. Pour créer un masque 2D, on peut utiliser les lignes de commandes suivantes :

In [ ]:
np.random.seed(seed=1)
n1,n2=np.shape(im)
r=np.random.rand(n1,n2) #pour créer le masquage
M=(r<0.5)
M = M*1.0

On peut bien entendu faire varier la proportion de l'image qu'on masque.

On masque l'image de la manière suivante :

In [ ]:
temp=M*im
pn.Row(hv.Image(im).opts(cmap='gray',width=400,height=400),hv.Image(temp).opts(cmap='gray',width=400,height=400))

### Initilisation par un filtre médian.

Proposer et tester un algorithme de Filtre médian qui réalise un inpainting primaire en rempalçant les pixels manquants par la mediane des coefficients observés sur des carrés de taille vois*vois

In [ ]:
def FiltreMedian(im,masque,vois):
    imrec=np.copy(im)
    n1,n2=np.shape(im)
    K1=int(np.floor(n1/vois))
    K2=int(np.floor(n2/vois))
    #print(np.arange(K1))
    #print(np.arange(vois))
    for k1 in np.arange(K1): #Boucle sur le nombre de carré
        k1=8*k1 #on fait fois 8 pour ne pas reboucler sur un carré déjà complété
        for k2 in np.arange(K2):
            k2=8*k2 #idem pour ne pas reboucler sur un carré déjà completé
            median=[] #pour calculer la médiane dans un carré
            for kk1 in np.arange(vois): #Boucle pour calculer la médiane sur le carré de taille VOISxVOIS
                for kk2 in np.arange(vois):
                    if imrec[k1+kk1,k2+kk2] != 0: #si jamais le pixel n'est pas masqué on append sa valeur dans le calcul de la médiane 
                        median.append(imrec[k1+kk1,k2+kk2])
            med = np.median(median) #calcul de la médiane
            #print(med)
            for kk1 in np.arange(vois): #Boucle pour remplacer la médiane calculée ci-dessus sur les pixels à valeurs nuls 
                for kk2 in np.arange(vois):
                    if imrec[k1+kk1,k2+kk2] == 0:
                        imrec[k1+kk1,k2+kk2]=med
        
    return(imrec)

In [ ]:
imrec2=FiltreMedian(temp,M,8) #on prend vois=8 pour la taille des carrés
hv.Raster(temp).opts(cmap='gray',xaxis=None,yaxis=None,width=350,height=350)\
+hv.Raster(imrec2).opts(cmap='gray',xaxis=None,yaxis=None,width=350,height=350) 
#on affiche les deux images masquée et reconstruite

### Retour sur l'algorithme FB

Proposer une fonction qui calcule le gradient de la fonction $f(x)=\frac{1}{2}\Vert Mx-y\Vert^2$ :

In [ ]:
def GradientInpainting(x,y,M): #pour calculer le gradient de f(x)
    return M*(M*x - y) #normalement on devrait avoir Mtranspose mais ici M est symétrique donc on peut laisser M tout court !

Pour quelle valeur $L$ ce gradient est-il $L$-Lipschitz ? 

**Pour L=1 le grandient est Lipshcitz**

Les fonctions suivantes permettent d'évaluer l'opérateur proximal de la fonction 
$g(x)=\lambda \Vert Tx\Vert_1$ où $T$ est une transformée orthogonale en ondelettes ou une transformée en ondelettes à trou, c'est-à-dire invariante par translation. Vous noterez qu'une transformée en Ondelettes de Daubechies db2 à été choisie ici par défaut mais que ce ci peut bien entendu être changé.

In [ ]:
def SeuillageDouxOndelettes(I,wave,Seuil): #Un seuillage doux équivaut ici au calcul de l'opérateur proximal de la fonction g(x), on réutilise la même fonction que dans le TP Debruitage Ondelettes 2D
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    temp=pywt.threshold(arr,Seuil,mode='soft')
    test=pywt.unravel_coeffs(temp, coeff_slices, coeff_shapes, output_format='wavedecn')
    Irec=pywt.waverecn(test, wave,mode='per')
    return Irec
def Normel1Ondelettes(I,wave): #fonction qui calcule la norme 1 de Tx donc nous donne une valeur explicite de la fonction g(x)
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    norml1=sum(np.abs(arr))
    return norml1

In [ ]:
wave='haar'
imrec=SeuillageDouxOndelettes(im,wave,3)

Proposer et tester un code d'inpaiting utilisant Forward-Backward minimisant la fonctionnelle (1) et la tester.

On pensera à cliper le résultat entre 0 et 255.

In [ ]:
def ForwardBackwardInpaintingv2(y,M,step,lam,Niter,wave):
    #z=y
    z=np.zeros(y.shape)
    F=[]
    for i in range(Niter): #on boucle sur un nombre d'itérations arbitraire pour minimiser la fonction (il faut juster garder le même en FB qu'en FISTA pour que ça soit comprabale)
        z = SeuillageDouxOndelettes(z-step*GradientInpainting(z,y,M),wave,lam) #lambda représente ici le seuil
        F.append(1/2 * np.linalg.norm(M*z-y,2)**2 + lam * Normel1Ondelettes(z,wave)) #on append la valeur de la fonction minimisée après une itération de l'algo de FB
    return np.clip(z,0,255),F

In [ ]:
y=M*im
step=3
#step=20
lam=10
Nb_iter=100
wave='haar'
Ta=350
imrec,f=ForwardBackwardInpaintingv2(y,M,step,lam,Nb_iter,wave)
pn.Row(hv.Image(im).opts(cmap='gray',width=Ta,height=Ta),\
       hv.Image(y).opts(cmap='gray',width=Ta,height=Ta)\
       ,hv.Image(imrec).opts(cmap='gray',width=Ta,height=Ta))

In [ ]:
hv.Curve(f).opts(xlabel='Nombre d\'itérations', ylabel='F(x)')

On voit que si le 'step' est très grand (ex : 20) notre fonction coût explose.

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']

Proposer un dashboard permettant une exploration numérique de la fonction précédente. On pourra ainsi observer que les petites valeurs de $\lambda$ donnent de meilleurs résulats. En théorie, si les données observées ne sont pas bruitées, il faudrait prendre une valeur de $\lambda$ aussi petite que possible. Vous pouvez le remarquer si vous affichez le PSNR de l'image reconstruite et que vous faites un nombre suffisant d'itérations.

Le problème d'un tel choix est que cela ralentit considérablement l'algorithme. Si les données sont bruitées, la valeur optimale de $\lambda$ est proportionnelle à lécart type du bruit.  

Ainsi, vous aurez intérêt à choisir une valeur faible de $\lambda$ mais pas trop petite pour assurer que votre algorithme est proche de la convergence.

In [ ]:
class FBInpaint(param.Parameterized): #on crée une classe FB Inpaint pour mieux visualiser les résultats de notre algorithme en modifiant les paramètres
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    Niter = param.Integer(30,bounds=(0,100))
    lam = param.Number(10,bounds=(1,30))
    step = param.Number(0.9,bounds=(0.5,4))
    masquage = param.Number(0.5,bounds=(0.1,1))
    def view(self):
        I=imagesRef[self.image]
        M = np.random.rand(*I.shape) #on crée un masque
        M[M<self.masquage] = 0
          
        imasqu=M*I #on recrée l'image masquée
        
        
        Irec,F=ForwardBackwardInpaintingv2(imasqu, M, self.step,self.lam,self.Niter,self.wave) 
        
        #Calcul des PSNRs des images masquées et reconstruites par rapport à l'image classique
        PSNR_masq = PSNR(I,imasqu)
        PSNR_rec = PSNR(I,Irec)
        
        return pn.Row(hv.Image(I).opts(title=f'Image classique',cmap='gray'), 
                      hv.Image(imasqu).opts(title=f'Image masquée, PSNR={PSNR_masq:.2f}', cmap='gray'), 
                      hv.Image(Irec).opts(title=f'Image reconstruite, PSNR={PSNR_rec:.2f}', cmap='gray'))
        
        

In [ ]:
fbinpaint= FBInpaint()
pn.Row(fbinpaint.param,fbinpaint.view)

Le dashboard permet de faire varier les différents paramètres de notre algorithme, en particulier le pas que l'on a défini en tant que $h$ dans notre modèle. On voit notamment qu'en prenant un $h$ plus grand que 1 notre image est, comme prévu, grisée car on ne la démasque pas (cela est dû àl'explosion de la fonction coût). 

On remarque que si step > 1 alors on a le PSNR qui explose vers +inf et on a plus d'image

# Accélération de Nesterov, FISTA.

Yuri Nesterov a proposé dans les années 1980 plusieurs méthodes permettant l'accélération de la descente de gradient explicite. Nous allons nous intéresser à une en particulier, celle qui est détaillée dans le polycopié de cours. Si vous faites des recherches bibliographiques, soyez conscients que sous le terme "Accélération de Nesterov" peuvent se cacher différentes méthodes, par interpolation ou extrapolation, spécifiques aux fonctions fortement convexes ou pas, s'appliquant à une descente de gradient simple ou à des fonctions composites. De plus nous n'utiliserons pas les paramètres historiques proposés par Nesterov, mais ceux qu'Antonin Chambolle et moi même avons proposé en 2014 pour en améliorer la vitesse et la convergence.

y=np.array([-2,-3,1,0,1,0,-2,-3])
print(ShannonEntropy(y))
L'accélération de la descente de gradient proposée Yurii Nesterov en 1984 et adapatée à FB sous le nom de FISTA par Beck et Teboulle en 2009 est d'une mise en oeuvre très simple.

Il suffit d'effectuer la descente de gradient en un point décalé de $x_n$ avec un pas $h<\frac{1}{L}$ (attention cette borne est plus contraignante que pour la descente de gradient classique) :
$$x_{n+1}=T(x_n+\alpha_n(x_n-x_{n-1}))$$ 
où la suite $\alpha_n$ est bien choisie et $T$ st soit l'opérateur de descente de gradient explicite, soit l'opérateur associé au FB. On parle de méthode inertielle car cette méthode utilise un terme dit de "mémoire" ou inertiel qui exploite la dernière direction de descente.

Donnons quelques éléments clés importants :

1) Le choix original de Nesterov pour la suite $\alpha_n$ ets le suivant :
\begin{equation}
\alpha_n=\frac{t_n-1}{t_{n+1}}\text{ avec }t_1=1\text{ et }t_{n+1}=\frac{1+\sqrt{1+t_n^2}}{2}
\end{equation}
2) Pour ce choix on a 
$$F(x_n)-F(x^*)\leqslant \frac{2\Vert x_0-x^*\Vert^2}{hn^2}$$
3) On peut prendre plus simplement $\alpha_n=\frac{n-1}{n+a-1}$ avec $a>3$ (c'est ce que vous programmerez) dans ce cas, on a $$F(x_n)-F(x^*)\leqslant \frac{(a-1)^2\Vert x_0-x^*\Vert^2}{2h(n+a)^2}$$ et en plus $F(x_n)-F(x^*)=o\left(\frac{1}{n^2}\right)$ et on a convergence de la suite $(x_n)_{n\geqslant 1}$. On peut noter que dans ce cas, la première étape est une simple sans inertie ($\alpha_1=0$) et donc $x_1=T(x_0)$. L'inertie apparait pour le calcul de $x_2$. Le choix originel de Nesterov est très proche du choix $a=3$. 

4) Dans le cas d'un fonction composite, on parle de l'algotithme FISTA (Fast Iterative Soft Shrinckage Algorithm) car si $g$ est une norme $\ell_1$, l'opérateur proximal se réduit à un seuillage doux. Mais FISTA est le nom générique proposé par Beck et Teboulle en 2009, ce peut être trompeur.

5) La suite de terme général $F(x_n)-F(x^*)$ n'est pas nécessairemment décroissante comme dans le cas de FB ou de la descente de gradient. Les bornes données précédemment sont des bornes mais ne reflètent pas la décroissance réelle. Dans la pratique vous verrez que FISTA est quand même plus rapide que FB.


Proposer un algorithme d'inpainting consistant à minimiser la fonctionnelle (1) en utilisant FISTA. 

In [ ]:
def FISTAInpainting(y,M,step,lam,Niter,wave,alpha):
    z=np.zeros(y.shape)
    z_old=z
    F=[] #F=[1/2 * np.linalg.norm(M*z-y,2)**2 + lam * Normel1Ondelettes(z,wave)]
    for k in np.arange(0,Niter):
        #on garde la formule 3 car on a bien a>4 car elle est plus simple 
        temp=z+(k/(k+alpha))*(z-z_old) #ici alpha = a initial donc 4 tous le temps
        z_old=z
        z= SeuillageDouxOndelettes(temp-step*GradientInpainting(temp,y,M),wave,lam) #on applique la fonction seuillage doux en appliquant bien la fonction qui calcule le gradient de la norme 
        F.append(1/2 * np.linalg.norm(M*z-y,2)**2 + lam * Normel1Ondelettes(z,wave))
    return np.clip(z,0,255),F

In [ ]:
imrec,fista=FISTAInpainting(y,M,step,lam,Nb_iter,wave,4) 
#on garde le même lambda pour que l'on minimise bien la même fonction qu'en FB pour qu'on puisse comparer
#De même on garde le même NB_iter dans les deux fonctions également !
pn.Row(hv.Image(im).opts(cmap='gray',width=Ta,height=Ta),\
       hv.Image(y).opts(cmap='gray',width=Ta,height=Ta)\
       ,hv.Image(imrec).opts(cmap='gray',width=Ta,height=Ta))

In [ ]:
#On crée de nouveau une classe pour l'affichage
class FISTAInpaint(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    Niter = param.Integer(10,bounds=(0,300))
    lam = param.Number(10,bounds=(1,30))
    step = param.Number(0.9,bounds=(0.5,4))
    masquage = param.Number(0.5,bounds=(0.1,1)) #: pas utile dans notre fonction
    alpha = param.Number(3,bounds=(1,10))
    def view(self):
        I=imagesRef[self.image]
        imasqu=M*I #on recrée l'image masquée
        
        Irec2,F2=FISTAInpainting(I,self.masquage,self.step,self.lam,self.Niter,self.wave,self.alpha)
        
        #Calcul des PSNRs des images masquées et reconstruites par rapport à l'image classique
        PSNR_masq = PSNR(I,imasqu)
        PSNR_rec = PSNR(I,Irec2)
        
        return pn.Row(hv.Image(I).opts(title=f'Image classique',cmap='gray'), 
                      hv.Image(imasqu).opts(title=f'Image masquée, PSNR={PSNR_masq:.2f}', cmap='gray'), 
                      hv.Image(Irec2).opts(title=f'Image reconstruite, PSNR={PSNR_rec:.2f}', cmap='gray'))
        

In [ ]:
fistainpaint= FISTAInpaint()
pn.Row(fistainpaint.param,fistainpaint.view)

In [ ]:
hv.Curve(fista).opts(xlabel='Nombre d\'itérations', ylabel='F(x)')

**On veut désormais vérifier que l'algo FISTA est bien + rapide que l'algo FB, du coup on affiche les deux courbes sur le même graphique :** 

In [ ]:
curve1 = hv.Curve(f, label='FB')
curve2 = hv.Curve(fista, label='FISTA')

curves = curve1 * curve2 #pour afficher les deux courbes sur le même plot

curves.opts(legend_position='right', width=600, height=400,xlabel='Nombre d\'itérations', ylabel='F(x)')

On voit bien que l'algo FISTA minimise la fonction plus rapidement : une 15aine d'itérations contre plus de 40 pour FB.

On remarque une petite vaguelette sur la courbe FISTA car contrairement au FB ce n'est pas un algo de descente uniforme.

## Pour aller plus loin (facultatif et à titre d'information)

Comme pour le débruitage, on peut utiliser des bases d'ondelettes translatées pour améliorer la qualité de l'inpainting. Dans ce cas, il ne faut évidemment pas recalculer l'inpainting depuis le début pour chaque version translatée, mais utiliser comme initialisation les versions inpaintées précédentes. 
Cette variante permet également de lisser les défauts et supprime des effets liés à la forme de l'ondelettes et donc les effets de blocs si on utilise les ondelettes de Haar.

## Débruitage par minimisation de la variation totale (Facultatif d'après Monsieur Nguyen)

Si l'image qu'on souhaite débruiter une image dont on sait qu'elle a une faible variation totale (norme $\ell_1$ du gradient), on peut estimer l'image en minimisant une fonctionnelle . Attention, on parle ici du gradient de l'image, pas d'une fonctionnelle qui associe une image à un réel. 
Le gradient d'une image est un champ de vecteur avec deux composantes, l'une verticale, l'autre horizontal qui se calcule de manière classique par différence finie. J'ai fait le choix dans les codes proposés d'une discrétisation du gradient et de la discrétisation de l'opérateur adjoint associé. En particulier, cette version n'est pas périodisée. Si vous voulez utiliser une autre implémentation du gradient discret, il est probable qu'il faille recoder l'opérateur 
adjoint.

Si on note $y=x^0+b$ les observations où $x^0$ est l'image à estimer et $b$ un bruit additif, cette fonctionnelle sera de la forme 
\begin{equation}\label{Primal}\tag{Primal}
\frac{1}{2}\Vert x-y\Vert^2+\lambda \Vert \nabla x\Vert_1
\end{equation}

12‖𝑥−𝑦‖2+𝜆‖∇𝑥‖1(Primal)Minimiser une telle fonctionnelle revient à caluler l'opérateur proximal de la fonction 
$g(x)=\lambda \Vert \nabla x\Vert_1$, or il n'existe pas de formule pour un tel opérateur. Ce qui implique aussi que réaliser un Forward Backward avec le terme quadratique comme terme différentiable ne peut pas fonctionner, il nécessite la connaissance de cet opérateur proximal.

Il existe cependant un moyen d'utiliser l'algorithme FB pour résoudre ce problème : on procède par dualisation.
Il n'est pas possible de détailler cette approche ici, je vous renvoie sur le Poly. La dualisation qui utilise la conjugué de Fenchel Rockafellar permet de montrer qu'on peut associer au problème précédent, le problème doptimisation suivant
\begin{equation}\label{Dual}\tag{Dual}
\min_{p}\frac{1}{2}\Vert div(p)+y\Vert^2+\iota_{\mathcal{B}_{\infty,\lambda}}(p)
\end{equation}
où $p$ est un champ de gradient et 
où $\iota_{\mathcal{B}_{\infty,\lambda}}$ est la boule $\ell_{\infty}$ de rayon $\lambda$ et où $div$ est l'opérateur 
de divergence. La présence de la divergence est ici due au fait que $-div$ est l'opérateur adjoint du gradient $\nabla$. La norme $\infty$ est ici présente car c'est la norme duale de la norme $/ell_1$.

Si $p^*$ est la solution du problème précédent alors $x^*=y+div(p^*)$ est la solution du problème initial.

L'avantage de ce second problème est qu'il peut être résolu par l'algorithme FB dans la mesure où le terme non différentiable est une indicatrice d'un ensemble sur lequel on sait projeter.

Les fonctions suivantes permettent de calculer le gradient et la divergence.
Tel que tout est codé ici, il s'agit d'une divergence négative, c'est à dire l'opposé de la divergence. A vous de voir si vous voulez tout remodifier. 

In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return y
def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

In [ ]:
def ProjGradBouleInf(g,l):
    gh=g[0]
    gv=g[1]
    temp=g
    p0=gh-(gh-l)*(gh>l)-(gh+l)*(gh<-l)
    p1=gv-(gv-l)*(gv>l)-(gv+l)*(gv<-l)
    temp[0]=p0
    temp[1]=p1
    return temp

Proposer une fonction Python qui résout le problème dual par Forward Backward

In [ ]:
def FBDenoisingTV(y,l,step,Niter):
    #ForwardBackwardInpaintingv2(y,M,step,lam,Niter,wave)
    return np.clip(x,0,255)

Tester ce code sur un exemple d'image bruitée par un bruit gaussien. L'image résultante subit un effet Cartoon. Pourquoi ? Faite varier les paramètres pour voir les effets des différents paramètres de l'algorithme.  
Le pas doit être maintenu en dessous de $1/8$ pour vérifier les conditions de Lipschitz du gradient.

In [ ]:
n1,n2=np.shape(im2)
Sigma=30
Noise=np.random.randn(n1,n2)
imb2=im2+Sigma*Noise
imrec2=FBDenoisingTV(imb2,25,0.1,2000)
pn.Row(hv.Image(im2).opts(cmap='gray',width=Ta,height=Ta),\
       hv.Image(np.clip(imb2,0,255)).opts(cmap='gray',width=Ta,height=Ta)\
       ,hv.Image(imrec2).opts(cmap='gray',width=Ta,height=Ta))



Créer le dashboard associé et tester l'algorithme sur différentes images et faites varier les paramètres.
Le dashboard doit renvoyer 3 images et 2 PSNR. Vous devrier observer un effet "cartoon" surtout pour de grandes valeurs du paramètre $\lambda$. 

In [ ]:
class DenoisingTVFB(param.Parameterized):
    Niter = param.Integer(100,bounds=(0,500))
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    lam = param.Number(9,bounds=(1,50))
    step = param.Number(0.1,bounds=(0.1,2))
    Sigma = param.Number(17,bounds=(1,100))
    

In [ ]:
denoisingTVFB=DenoisingTVFB()
pn.Row(denoisingTVFB.param,denoisingTVFB.view)

Que donnerait cette approche sur des images type "Cartoon" ? Tester.

# A faire ensuite...

1) Si ce n'est déjà fait, on peut accélérer le code en initialisant la suite par un seuillage simple en ondelettes...
On peut même essayer de comparer les deux versions.

2) Appliquer FISTA à ce même problème et réaliser le dashboard associé.

3) Réaliser un plan d'expériences pour comparer les différents choix de paramètres, uniquement pour la version FISTA. Il faut faire très attention ici car dès que le nombre d'itérations devient conséquent, chaque expérience prend un temps non négligeable. Assurez vous avant de lancer les XP numérique que le temps total est raisonnable.

4) Comparer cette méthode en terme de PSNR, pour les valeurs optimales des paramètres aux meilleurs méthodes par seuillage en ondelettes utilisant les translations.

## Pour aller plus loin (facultatif et à titre d'information)

### Plusieurs définitions de la norme de variation totale.

Il existe plusieurs définition de la norme de variation totale. Celle que nous utilisons ici est celle qui est aussi appelée anisotrope mais il existe aussi une variation totale isotrope :

https://en.wikipedia.org/wiki/Total_variation_denoising

L'utilisation la variation totale isotrope privilégie moins les axes horizontaux et verticaux. Ce choix induit des défaut qui sont plus homogène en fonctions des directions. La différence entre les deux approches est particulièrement visible sur les coins des objet, ou sur les bords de disques.

Pour passer en version isotrope, il faut changer de norme dans \eqref{Primal} ainsi... la définition de la boule $\ell_\infty$ et donc de la projection dans \eqref{Dual} sont légèrement différentes.  

Il faut savoir que cette petite finesse existe et bien être au clair sur les normes que l'on manipule.

### Débruitages d'images couleurs

On peut effectuer un débruitage similaire des images couleurs en effectuant un débruitage canal chromatique par canal chromatique. Mais il est aussi possible de travailler directement sur l'image couleur et de définir une norme de variation totale directement sur les 3 canaux, en sommant en sommant en chaque pixel la norme du vecteur gradient à 6 composantes (3 canaux, 2 directions). L'inétrêt de faire un calcul joint de la norme du gradient est de rendre cohérents les trois canaux de couleurs.

## A retenir.

Si vous avez une descente de gradient ou un Forward-Backward à implémenter, les accélérations de Nesterov sont très souvent efficaces et il est toujours rentable de tester et même de tester différentes valeurs de $a$. Il est à noter toutefois que théoriquement sur des fonctions fortement convexes, pour l'algorithme FB, la suite de terme général $F(x_n)-F(x^*)$ décroit au moins géométriquement ce qui n'est pas le cas de l'accélération de Nesterov présentée ici qui sont au mieux polynomiales.
Dans la pratique, pour un nombre raisonnable d'itérations, même dans ce cas, les accélérations de Nesterov sont largement compétitives.
Mais il en existe d'autres encore plus efficaces spécifiquement dédiés à ce cas.